## Introducción

En estos ejercicios vamos a trabajar con los datasets alojado en la ruta del curso **/Dia-05/datasets/XXVideos** representan el conjunto de estadísticas de vídeos de Youtube en los países de Canadá, USA y México.

Las columnas del dataframe significan lo siguiente:

1. `video_id` : Id del vídeo
2. `trending_date` : Fecha de registro de estadísticas
3. `title` : Título del vídeo
4. `channel_title` : Nombre del canal de Youtube
5. `category_id` : Id de la categoría del vídeo
6. `publish_time` : Fecha de publicación del vídeo
7. `tags` : Etiquetas del vídeo
8. `views` : Vistas del vídeo en la fecha 'trending_date'
9. `likes` : Likes del video en la fecha 'trending_date'
10. `dislikes` : Dislikes del video en la fecha 'trending_date'
11. `comment_count` : Número de comentarios en la fecha 'trending_date'
12. `thumbnail_link` : Link a la miniatura del vídeo
13. `comments_disabled` : Comentarios desactivados
14. `ratings_disabled` : Likes/Dislikes desactivados
15. `video_error_or_removed` : Video borrado o con error en subida
16. `description` : Descripción del vídeo

Ejecuta la siguiente celda para el dataframe con el contenido del dataset.

In [ ]:
import pandas as pd

ca_videos = pd.read_csv("../datasets/CAvideos.csv")
mx_videos = pd.read_csv("../datasets/MXvideos.csv", encoding='latin-1')
usa_videos = pd.read_csv("../datasets/USvideos.csv")

mx_videos

## Ejercicio 1

Imprime los tipos de datos de las columnas de los 3 dataframes. ¿Tienen los mismos tipos de datos?

In [ ]:
ca_videos.dtypes

In [ ]:
mx_videos.dtypes

In [ ]:
usa_videos.dtypes

## Ejercicio 2 

Renombra las columnas `trending_date` y `publish_time` a los nombres `statistics_date` y `publish_date` respectivamente para los 3 paises.

In [ ]:
ca_videos = ca_videos.rename(columns={'trending_date': 'statistics_date', 'publish_time': 'publish_date'})
usa_videos = usa_videos.rename(columns={'trending_date': 'statistics_date', 'publish_time': 'publish_date'})
mx_videos = mx_videos.rename(columns={'trending_date': 'statistics_date', 'publish_time': 'publish_date'})

## Ejercicio 3

Convierte las columnas `statistics_date` y `publish_date` de los 3 dataframe tipo de dato datetime64 con el formato **YYYY-MM-DD** para poder realizar funciones de agrupación y extracción de fechas.

In [ ]:
def transformar_fecha_estadisticas(df, column_name):
    df[column_name] = pd.to_datetime(pd.to_datetime(df[column_name], format='%y.%d.%m').dt.strftime('%Y-%m-%d'))

def transformar_fecha_publicacion(df, column_name):
    df[column_name] = pd.to_datetime(pd.to_datetime(df[column_name]).dt.strftime('%Y-%m-%d'))

# Llamadas a la función para cada DataFrame y columna necesaria
transformar_fecha_estadisticas(ca_videos, 'statistics_date')
transformar_fecha_publicacion(ca_videos, 'publish_date')

transformar_fecha_estadisticas(usa_videos, 'statistics_date')
transformar_fecha_publicacion(usa_videos, 'publish_date')

transformar_fecha_estadisticas(mx_videos, 'statistics_date')
transformar_fecha_publicacion(mx_videos, 'publish_date')

## Ejercicio 4

¿En que año se subieron más vídeos para cada país? 

**Ten en cuenta que puede haber más de una fila que compartan el mismo video (`video_id`)**

In [ ]:
df_unique_ids = ca_videos.drop_duplicates(subset='video_id')

df_unique_ids

In [ ]:
def max_videos_year(df, country):
    # Extraer el año de la columna publish_date
    df = df.drop_duplicates(subset = "video_id")
    df['publish_year'] = pd.to_datetime(df['publish_date']).dt.year
    
    # Agrupar los datos por año, contar el número de vídeos subidos 
    videos_agg = df.groupby(['publish_year']).size().idxmax()
    
    # Imprimir el resultado
    print(f"Año con más vídeos subidos para {country}:")
    print(videos_agg)

# Llamadas a la función para cada país
max_videos_year(ca_videos, "Canada")
max_videos_year(usa_videos, "USA")
max_videos_year(mx_videos, "Mexico")

## Ejercicio 5 

¿Qué mes del año 2017 tiene el record en visitas para el dataframe de vídeos de México?

In [ ]:
# Eliminamos duplicados para que no se sume todos los registros de un mismo vídeo. Nos quedamos con la última actualización que tenemos de un vídeo. Esas visitas son las que cuentan,
# ya que son las visitas finales que se tienen registradas sobre ese vídeo.

idx = mx_videos.groupby('video_id')['statistics_date'].idxmax()

mx_videos_unique = mx_videos.loc[idx]


print(mx_videos.shape)
print(mx_videos_unique.shape)

In [ ]:
mx_videos_unique['publish_year'] = pd.to_datetime(mx_videos_unique['publish_date']).dt.year

# Filtrar los datos para incluir solo los vídeos de 2017
mx_videos_2017 = mx_videos_unique[mx_videos_unique['publish_year'] == 2017]

# Extraer el mes de la columna 'publish_date'
mx_videos_2017['publish_month'] = mx_videos_2017['publish_date'].dt.month

# Agrupar por mes y sumar las visitas
monthly_visits_2017 = mx_videos_2017.groupby('publish_month')['views'].sum()

# Encontrar el mes con el mayor número de visitas
mes_max_visitas = monthly_visits_2017.idxmax()

mes_max_visitas

## Ejercicio 6

¿Qué dataframe de los 3 tiene más valores nulos entre todas sus columnas? Calcula el porcentaje de columnas nulas de los dataframes utilizando la función `apply`

In [ ]:
# Calcula el porcentaje de valores nulos en cada DataFrame
porcentaje_nulos_ca = ca_videos.apply(lambda col: (col.isnull().sum() / len(col)) * 100)
porcentaje_nulos_usa = usa_videos.apply(lambda col: (col.isnull().sum() / len(col)) * 100)
porcentaje_nulos_mx = mx_videos.apply(lambda col: (col.isnull().sum() / len(col)) * 100)

print(porcentaje_nulos_mx)
# Suma el porcentaje de valores nulos en todas las columnas de cada DataFrame
total_nulos_ca = porcentaje_nulos_ca.sum()
total_nulos_usa = porcentaje_nulos_usa.sum()
total_nulos_mx = porcentaje_nulos_mx.sum()

# Determina cuál DataFrame tiene más valores nulos entre todas sus columnas
max_nulos = max(total_nulos_ca, total_nulos_usa, total_nulos_mx)

if max_nulos == total_nulos_ca:
    print("El DataFrame de vídeos de Canadá tiene más valores nulos entre todas sus columnas.")
elif max_nulos == total_nulos_usa:
    print("El DataFrame de vídeos de USA tiene más valores nulos entre todas sus columnas.")
else:
    print("El DataFrame de vídeos de México tiene más valores nulos entre todas sus columnas.")

## Ejercicio 7

Reemplaza los valores nulos del dataframe con más valores nulos con los siguientes criterios:

- 1. Si la columna es de tipo int64 reemplaza el valor nulo por la moda de valores de esa columna. (La moda es el valor que más se repite dentro de los valores de la columna.)
- 2. Si la columna es de tipo string rellena cada valor nulo por el valor no nulo más próximo después de ese registro. (Estrategia de relleno)

In [ ]:
def imputar_valores_nulos(df):
    for column in df.columns:
        if df[column].dtype == 'int64':
            # Si la columna es de tipo int64, reemplazar los valores nulos por la moda
            moda = df[column].mode()[0]
            df[column].fillna(moda, inplace=True)
        elif df[column].dtype == 'object':
            # Si la columna es de tipo string, rellenar los valores nulos con la estrategia de relleno
            df[column].fillna(method='ffill', inplace=True)

# Imputar los valores nulos en el DataFrame con más valores nulos
imputar_valores_nulos(mx_videos)

## Ejercicio 8

¿Cuántos vídeos tienen audiciencia compartida entre los 3 dataframes? Es decir, que se hayan visto en USA, Canadá y México. 

**Puedes combinar los dataframes por la columna `video_id`**

In [ ]:
# Nos quedamos con la última actualización de la información de los vídeos para los 3 países.
idx = mx_videos.groupby('video_id')['statistics_date'].idxmax()
mx_videos = mx_videos.loc[idx]

idx = ca_videos.groupby('video_id')['statistics_date'].idxmax()
ca_videos = ca_videos.loc[idx]

idx = usa_videos.groupby('video_id')['statistics_date'].idxmax()
usa_videos = usa_videos.loc[idx]

# Realiza los joins entre los DataFrames por la columna 'video_id'
join_df = ca_videos.set_index('video_id').join(usa_videos.set_index('video_id'), lsuffix='_ca', rsuffix='_usa', how='inner').join(mx_videos.set_index('video_id'), lsuffix='_ca', rsuffix='_mx', how='inner')

# Encuentra el número de vídeos con audiencia compartida
num_videos_audiencia_compartida = join_df.shape[0]

print("El número de vídeos con audiencia compartida entre USA, Canadá y México es:", num_videos_audiencia_compartida)

## Ejercicio 9

¿Cuál es el vídeo más visto que se vió en los 3 países? ¿Y el que tuvo más likes?

In [ ]:
# Encontrar el vídeo más visto en los tres países
video_mas_visto = join_df['views'].idxmax()
max_views = join_df.loc[video_mas_visto, 'views']

# Encontrar el vídeo con más likes en los tres países
video_mas_likes = join_df['likes'].idxmax()
max_likes = join_df.loc[video_mas_likes, 'likes']

print("El vídeo más visto que se vio en los tres países es:", join_df.loc[video_mas_visto, 'title'], "con", max_views, "vistas.")
print("El vídeo con más likes que se vio en los tres países es:", join_df.loc[ video_mas_likes,'title'], "con", max_likes, "likes.")
